In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic              # mediapipe 姿態偵測
mp_drawing = mp.solutions.drawing_utils          # mediapipe 繪圖方法

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
   # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
   # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )  


In [ ]:
# KeyPoint如何連接
# mp_holistic.POSE_CONNECTIONS

In [ ]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)as holistic:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()
        # Make detection
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        #Draw Landmarks
        draw_styled_landmarks(image,results)
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()


In [ ]:
len(results.face_landmarks.landmark)

In [ ]:
draw_styled_landmarks(frame,results)

In [ ]:
plt.imshow(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))

In [ ]:
results.pose_landmarks.landmark[0] #.x .y .z .visibility
#提取第一個frame的landmark的各個數值

In [ ]:
pose=[]
for res in results.pose_landmarks.landmark:#存成array
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [ ]:
# 33組x,y,z,visibility攤平成一個大陣列有132個landmark
# pose.shape (33,4)攤平成(132,)
pose=np.array([[res.x,res.y,res.z,res.visibility]for res in results.pose_landmarks.landmark]).flatten()if results.pose_landmarks else np.zeros(132)
face=np.array([[res.x,res.y,res.z]for res in results.face_landmarks.landmark]).flatten()if results.face_landmarks else np.zeros(1404)  
# 若frame裡面沒有出現左手,會跳error,要加if返回
lh=np.array([[res.x,res.y,res.z]for res in results.left_hand_landmarks.landmark]).flatten()if results.left_hand_landmarks else np.zeros(21*3)  
rh=np.array([[res.x,res.y,res.z]for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)  


In [ ]:
len(results.face_landmarks.landmark)*3

In [6]:
def extract_keypoints(results):
    pose=np.array([[res.x,res.y,res.z,res.visibility]for res in results.pose_landmarks.landmark]).flatten()if results.pose_landmarks else np.zeros(33*4)
    face=np.array([[res.x,res.y,res.z]for res in results.face_landmarks.landmark]).flatten()if results.face_landmarks else np.zeros(468*3)  
    lh=np.array([[res.x,res.y,res.z]for res in results.left_hand_landmarks.landmark]).flatten()if results.left_hand_landmarks else np.zeros(21*3)  
    rh=np.array([[res.x,res.y,res.z]for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)  
    return np.concatenate([pose, face, lh, rh])

In [ ]:
extract_keypoints(results).shape

In [ ]:
len(results.left_hand_landmarks.landmark)

In [ ]:
lh.shape

In [ ]:
np.zeros(21*3)

In [ ]:
extract_keypoints(results).shape

In [7]:
DATA_PATH = os.path.join('MP_Data') # 導出數據的路徑,numpy arrays
actions = np.array(['hello', 'thanks', 'iloveyou']) # 要偵測的動作種類
no_sequences = 30    # 30個影片的data
sequence_length = 30 # 每個影片長度為30 frames
start_folder = 30    #

In [8]:
for action in actions: # 建立Dataset的資料夾
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [ ]:
result_test=extract_keypoints(results)

In [ ]:
np.save('0',result_test)

In [ ]:
np.load('0.npy')

In [9]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)as holistic:
    for action in actions: # 動作循環,loop actions
        for sequence in range(no_sequences): # 影片循環30次,loop video
            for frame_num in range(sequence_length): # loop frames
        
                ret, frame = cap.read()# Read feed
                
                image, results = mediapipe_detection(frame, holistic)# Make detection
                
                draw_styled_landmarks(image,results) #Draw Landmarks
                
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA) #指定字體、大小、顏色、線寬
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                cv2.imshow('OpenCV Feed', image)

                keypoints = extract_keypoints(results) # 所有關鍵點攤平成array
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):# Break gracefully
                    break
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

資料預處理和建立標籤及特徵

In [10]:
import tensorflow as tf

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils.np_utils import to_categorical

In [12]:
label_map = {label:num for num, label in enumerate(actions)}

In [13]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [14]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
np.array(sequences).shape

In [ ]:
np.array(labels).shape

In [15]:
X = np.array(sequences)
y = to_categorical(labels).astype(int) # one-hot encoding

In [ ]:
labels

In [ ]:
y

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
y_train.shape

LSTM

In [17]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import LSTM, Dense
from tensorflow.python.keras.callbacks import TensorBoard

In [18]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir) #tensorboard callback

In [19]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [20]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
__________________________________________________

In [23]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])

Epoch 1/500
3/3 [==============================] - 5s 2s/step - loss: 18.9779 - categorical_accuracy: 0.3059
Epoch 2/500
3/3 [==============================] - 4s 1s/step - loss: 19.8388 - categorical_accuracy: 0.3294
Epoch 3/500
3/3 [==============================] - 4s 1s/step - loss: 10.9518 - categorical_accuracy: 0.3294
Epoch 4/500
3/3 [==============================] - 3s 1s/step - loss: 8.4741 - categorical_accuracy: 0.4471
Epoch 5/500
3/3 [==============================] - 4s 1s/step - loss: 15.2610 - categorical_accuracy: 0.2706
Epoch 6/500
3/3 [==============================] - 3s 1s/step - loss: 21.1081 - categorical_accuracy: 0.3176
Epoch 7/500
3/3 [==============================] - 3s 1s/step - loss: 18.1977 - categorical_accuracy: 0.4000
Epoch 8/500
3/3 [==============================] - 3s 1s/step - loss: 20.6645 - categorical_accuracy: 0.2471
Epoch 9/500
3/3 [==============================] - 3s 1s/step - loss: 8.4096 - categorical_accuracy: 0.4353
Epoch 10/500
3/3 [===